In [1]:
import os
import sys
import torch
import numpy as np
from google.colab.patches import cv2_imshow
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as F
import nibabel as nib
import matplotlib.pyplot as plt
import configparser
import pandas as pd

In [ ]:
!git clone https://github.com/msskzx/unimatch
dir_path = '/content/unimatch/more-scenarios/medical/'
os.chdir(dir_path)

# ACDC Dataset

## Download Test Dataset

In [ ]:
!wget --output-document=data.zip https://humanheart-project.creatis.insa-lyon.fr/database/api/v1/folder/6372203a73e9f0047faa117e/download
!unzip data.zip
!rm data.zip

## Test One Image Only

Inference on one image only, by providing the link of the patient folder

In [ ]:
!wget --output-document=data.zip https://humanheart-project.creatis.insa-lyon.fr/database/api/v1/folder/63721d7073e9f0047faa052a/download
!unzip data.zip
!rm data.zip
!mkdir testing
!mv patient001/ testing/patient001

# Model

In [ ]:
!pip install monai

In [5]:
from model.unet import UNet
from torch.optim import SGD

In [6]:
checkpoint = torch.load('exp/acdc/unimatch/unet/7/best.pth')
model = UNet(in_chns=1, class_num=4)

In [7]:
model_checkpoint = {k.replace('module.', ''): v for k, v in checkpoint['model'].items()}
model.load_state_dict(model_checkpoint)

<All keys matched successfully>

# Testing

In [12]:
from util.data_utils import get_acdc,convert_masks

# test dataloader
acdc_data, _, _ = get_acdc('testing', input_size=(224, 224, 1))
acdc_data[1] = convert_masks(acdc_data[1])
acdc_data[0] = np.transpose(acdc_data[0], (0, 3, 1, 2)) # for the channels
acdc_data[1] = np.transpose(acdc_data[1], (0, 3, 1, 2)) # for the channels
acdc_data[0] = torch.Tensor(acdc_data[0]) # convert to tensors
acdc_data[1] = torch.Tensor(acdc_data[1]) # convert to tensors
acdc_data = TensorDataset(acdc_data[0], acdc_data[1])
test_loader = DataLoader(acdc_data, batch_size=1, num_workers=2)
print(len(test_loader))

1076


## Prediction and Visualization

In [13]:
def evaluate_model(model, dataloader):
    device = torch.device("cuda")
    model.eval()
    model = model.to(device)
    patient_id = 101
    slice_id = 1
    i = 0
    #scores = pd.DataFrame(columns=['patient_id', 'slice_id', 'dice_mean', 'dice_lv', 'dice_rv', 'dice_myo'])
    scores = pd.DataFrame(columns=['patient_id', 'dice_mean'])
    dice_class = [0] * 3

    for img, mask in dataloader:
        img, mask = img.to(device), mask.to(device)

        with torch.no_grad():
            pred = model(img)

        pred = pred.argmax(dim=1).unsqueeze(0)

        # compute dice
        for cls in range(1, 4):
                    inter = ((pred == cls) * (mask == cls)).sum().item()
                    union = (pred == cls).sum().item() + (mask == cls).sum().item()
                    dice_class[cls-1] += 2.0 * inter / union

        dice_class = [dice * 100.0 / len(dataloader) for dice in dice_class]
        dice_mean = sum(dice_class) / len(dice_class)

        scores.loc[i] = {
                'patient_id': patient_id,
                #'slice_id': slice_id % 10 + 1,
                'dice_mean': dice_mean,
                #'dice_lv': dice_lv,
                #'dice_rv': dice_rv,
                #'dice_myo': dice_myo
            }
        if slice_id == 40:
          patient_id += 1
          slice_id = 0
        slice_id += 1
        i+= 1

    return scores

In [14]:
scores = evaluate_model(model, test_loader)

In [15]:
scores

,patient_id,dice_mean
0,101,0.000868
1,101,0.001006
2,101,0.001260
3,101,0.000696
4,101,0.000914
...,...,...
1071,127,0.000167
1072,127,0.000167
1073,127,0.000241
1074,127,0.000266


## Save Results

So it could be used for further analysis at anytime without infering again.

In [16]:
!mkdir results

In [17]:
export_path = 'results/unimatch_acdc_scores.csv'
scores.to_csv(export_path, index=False)
print(f"The scores have been saved to {export_path}")

The scores have been saved to results/unimatch_acdc_scores.csv
